In [13]:
import pandas as pd  # Veri işleme ve analiz için Pandas kütüphanesi
import numpy as np  # Sayısal işlemler için NumPy kütüphanesi
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern # Gri tonlama eş zamanlı matrisler ve LBP hesaplamak için Scikit-image kütüphanesi
from skimage import io, color, img_as_ubyte  # Görüntü işleme için Scikit-image kütüphanesi
import os  # Dosya yolları ve dizin işlemleri için os modülü
import cv2  # OpenCV kütüphanesi (görüntü işleme ve bilgisayarlı görü için)

In [14]:
df = pd.DataFrame(columns=['ContrastFeature', 'DissimilarityFeature', 'HomogeneityFeature', 'EnergyFeature',
                           'CorrelationFeature', 'ASMFeature', 'LBPFeature'])
# Boş bir DataFrame oluşturma

In [15]:
matrix1 = [] # Gri tonlama eş zamanlı matrisleri depolamak için boş bir liste oluşturma

In [16]:
path_of_images = r"C:\Users\User\Downloads\Odev_3\apple" # Görüntülerin bulunduğu dizini ayarlama
list_of_images = os.listdir(path_of_images) # Dizindeki tüm görüntülerin listesini alma, her görüntünün döngü içinde tek tek okunması ve işlenmesini sağlayacaktır. 

for image in list_of_images: # Dizindeki tüm görüntüleri tek tek işlemek için kullanılmıştır
    img = cv2.imread(os.path.join(path_of_images, image)) # Görüntüyü okuma
    gray = color.rgb2gray(img) # Görüntüyü gri tonlamaya çevirme
    image = img_as_ubyte(gray) # Gri tonlama eş zamanlı matrisi hesaplama

    # her görüntüyü 0 ile 255 arasındaki 16 eşit parçaya bölmek için kullanılmıştır. 
    # Bu, gri tonlama eş zamanlı matrislerinin hesaplanmasında gereklidir.
    bins = np.array([0, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240, 255])
    inds = np.digitize(image, bins)

    # Her görüntü için gri tonlama eş zamanlı matrisini hesaplamak için kullanılmıştır. 
    # Gri tonlama eş zamanlı matrisleri, görüntüdeki farklı yönlerde ve mesafelerde bulunan pikseller arasındaki benzerlikleri ve farklılıkları ölçmek için kullanılır.
    max_value = inds.max() + 1
    matrix_coocurrence = graycomatrix(inds, [1], [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4], levels=max_value,
                                      normed=False, symmetric=False)
    
    # LBP'yi uygulamadan önce görüntüyü tam sayı türüne dönüştür
    image_int = img_as_ubyte(gray)

    # Yerel İkili Model (LBP) özelliğini hesapla
    radius = 1
    n_points = 8 * radius
    lbp = local_binary_pattern(image_int, n_points, radius, method='uniform')

    matrix1.append((matrix_coocurrence, lbp))


In [17]:
# Her görüntü için hesaplanan özelliklerin depolanacağı boş listeler oluşturmak için kullanılmıştır. 
# Bu özellikler, görüntünün özelliklerini temsil eder ve görüntü tanıma, sınıflandırma ve diğer uygulamalarda kullanılabilir.
CF = []
DF = []
HF = []
EF = []
COR = []
ASM = []
LBP = []

In [18]:
# her görüntü için aşağıdaki özellikleri hesaplayan fonksiyonları tanımlamak için kullanılmıştır:
# Kontrast: Piksellerin parlaklıkları arasındaki farklılıkları ölçer.
# Farklılık: Piksellerin parlaklıklarına göre uzaklıklarını ölçer.
# Homojenlik: Piksellerin benzerlik derecesini ölçer.
# Enerji: Piksellerin parlaklıkları arasındaki varyansı ölçer.
# Korelasyon: Piksellerin parlaklıkları arasındaki ilişkiyi ölçer.
# ASM: Piksellerin parlaklıkları arasındaki varyansı ve korelasyonu birleştiren bir özelliktir.

for matrix, lbp in matrix1:
    def contrast_feature(matrix):
        contrast = graycoprops(matrix)
        return list(contrast)

    def dissimilarity_feature(matrix):
        dissimilarity = graycoprops(matrix)
        return list(dissimilarity)

    def homogeneity_feature(matrix):
        homogeneity = graycoprops(matrix)
        return list(homogeneity)

    def energy_feature(matrix):
        energy = graycoprops(matrix)
        return list(energy)

    def correlation_feature(matrix):
        correlation = graycoprops(matrix)
        return list(correlation)

    def asm_feature(matrix):
        asm = graycoprops(matrix)
        return list(asm)
        
    def lbp_feature(lbp):
        lbp_hist, _ = np.histogram(lbp, bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
        return lbp_hist.mean()
    
# Her görüntü için hesaplanan özelliklerin ortalamalarını hesaplamak ve listelere eklemek için kullanılmıştır.
    CF.append(np.mean(contrast_feature(matrix)))
    DF.append(np.mean(dissimilarity_feature(matrix)))
    HF.append(np.mean(homogeneity_feature(matrix)))
    EF.append(np.mean(energy_feature(matrix)))
    COR.append(np.mean(correlation_feature(matrix)))
    ASM.append(np.mean(asm_feature(matrix)))
    LBP.append(lbp_feature(lbp))

In [19]:
# Hesaplanan özellikleri DataFrame'e eklemek için kullanılmıştır.
Features = [CF, DF, HF, EF, COR, ASM, LBP]
for i, j in zip(df.columns, Features):
    df[i] = j

In [20]:
# DataFrame'e görüntülerin indeksini eklemek için kullanılmıştır. Bu, görüntülerin daha kolay takip edilmesini sağlar.
df.index = range(1, len(df) + 1)
df.index.name = 'Image ID'

In [21]:
df

,ContrastFeature,DissimilarityFeature,HomogeneityFeature,EnergyFeature,CorrelationFeature,ASMFeature,LBPFeature
Image ID,,,,,,,
1,1.340962,1.340962,1.340962,1.340962,1.340962,1.340962,3564.0
2,0.786452,0.786452,0.786452,0.786452,0.786452,0.786452,4000.0
3,0.435019,0.435019,0.435019,0.435019,0.435019,0.435019,4340.0
4,0.884693,0.884693,0.884693,0.884693,0.884693,0.884693,3240.0
5,0.355574,0.355574,0.355574,0.355574,0.355574,0.355574,5340.0
...,...,...,...,...,...,...,...
196,2.226111,2.226111,2.226111,2.226111,2.226111,2.226111,3420.0
197,1.253338,1.253338,1.253338,1.253338,1.253338,1.253338,4000.0
198,0.257872,0.257872,0.257872,0.257872,0.257872,0.257872,4590.0


In [22]:
df.to_excel('Feature_Extraction_from_Multiple_Apple_Image.xlsx') #  Hesaplanan özellikleri Excel dosyasına kaydetmek için kullanılmıştır

In [25]:
!pip install arff

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for arff: filename=arff-0.9-py3-none-any.whl size=4965 sha256=4104382c1ed32f10671d2f8891dd6cbf715ac718b785f0607c0ba0fff131e72f
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\26\1a\03\6b4ff8fbe4f80c0d2455fb557255fddf3addac1096d10120c2
Successfully built arff


In [26]:
import arff
arff.dump('Feature_Extraction_from_Multiple_Apple_Image.arff', 
          df.values,
          relation = 'Feature_Extraction_from_Multiple_Apple_Image',
          names = df.columns)